In [1]:
import seaborn as sns

from rcv_distribution import *

In [2]:
# Get the consistency points for the election
points = get_consistency_points("../rcv_elections_database/classic/Maine_11062018_CongressionalDistrict2.csv")

points

/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_learning/rcv_dimensionality.py:170: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)


{0.0: 89669,
 0.47423947404876676: 11600,
 1.4136378554535023: 579,
 3.0: 51673,
 2.192155473835831: 948,
 0.3167198370337414: 2801,
 0.41511902958066277: 2748,
 0.75: 3337,
 0.5333599185168707: 398,
 2.0540728885103365: 4401,
 0.5135182221275841: 2767,
 0.6317591110637921: 652,
 1.5027386629065809: 1087,
 1.2668793481349656: 2180,
 2.2905546663827523: 1131,
 1.7001595111012242: 291,
 2.03379555531896: 181,
 1.5819186221650163: 290,
 1.3053178147119375: 179,
 1.5417995925843535: 68,
 1.4636777332288085: 528}

In [3]:
import numpy as np
from scipy.stats import kurtosis, skew

# Get the consistency points for the election
points = get_consistency_points("../rcv_elections_database/classic/Maine_11062018_CongressionalDistrict2.csv")

# Create a list of data points
data_points = []
for key, value in points.items():
    data_points.extend([key] * value)

# Convert to numpy array
data_points = np.array(data_points)

# Calculate skewness and kurtosis
g = skew(data_points)
k = kurtosis(data_points)

# Calculate bimodality coefficient
n = len(data_points)
BC = (g**2 + 1) / (k + 3 * (n-1)**2 / ((n-2) * (n-3)))

BC

0.9173969066559533

In [15]:
import numpy as np
from scipy.stats import kurtosis, skew
import glob
import csv

# Open the output CSV file
with open('bimodality_results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the header
    writer.writerow(['filename', 'bimodality'])
    
    # Search for CSV files in the directory and its subdirectories
    csv_files = glob.glob("../rcv_elections_database/**/*.csv", recursive=True)

    for idx, file_path in enumerate(csv_files):
        # Get the consistency points for the election
        points = get_consistency_points(file_path)
        
        # Create a list of data points
        data_points = []
        for key, value in points.items():
            data_points.extend([key] * value)

        # Convert to numpy array
        data_points = np.array(data_points)

        # Calculate skewness and kurtosis
        g = skew(data_points)
        k = kurtosis(data_points)

        # Calculate bimodality coefficient
        n = len(data_points)
        bimodality = 0.0
        try:
            bimodality = (g**2 + 1) / (k + 3 * (n-1)**2 / ((n-2) * (n-3)))
        except:
            pass

        # Write the filename and bimodality coefficient to the CSV
        writer.writerow([file_path, bimodality])

        # Flush the output after every 10 runs
        if idx % 10 == 9:
            csvfile.flush()

ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
# Get the consistency points for the election
points = get_consistency_points("../rcv_elections_database/classic/Maine_11062018_CongressionalDistrict2.csv")

# Prepare data for histogram
data_list = [x for x, count in points.items() for _ in range(count)]

# Plot histogram
plt.figure(figsize=(5, 3))
plt.hist(data_list, bins=50, density=True, alpha=0.7)
plt.title('Histogram of Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

# Plot kernal density estimation
plt.figure(figsize=(5, 3))
sns.kdeplot(data_list, fill=True)
plt.title('Kernel Density Estimation of Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

In [ ]:
# Get the consistency points for the election
points = get_consistency_points("../rcv_elections_database/classic/Alaska_11082022_USRepresentative.csv")

# Prepare data for histogram
data_list = [x for x, count in points.items() for _ in range(count)]

# Plot histogram
plt.figure(figsize=(5, 3))
plt.hist(data_list, bins=50, density=True, alpha=0.7)
plt.title('Histogram of Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

# Plot kernal density estimation
plt.figure(figsize=(5, 3))
sns.kdeplot(data_list, fill=True)
plt.title('Kernel Density Estimation of Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

In [ ]:
# Get the consistency points for the election
points = get_consistency_points("../rcv_elections_database/classic/Alaska_08162022_HouseofRepresentativesSpecial.csv")

# Prepare data for histogram
data_list = [x for x, count in points.items() for _ in range(count)]

# Plot histogram
plt.figure(figsize=(5, 3))
plt.hist(data_list, bins=50, density=True, alpha=0.7)
plt.title('Histogram of Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

# Plot kernal density estimation
plt.figure(figsize=(5, 3))
sns.kdeplot(data_list, fill=True)
plt.title('Kernel Density Estimation of Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

In [ ]:
# Get the consistency points for the election
points = get_consistency_points("../rcv_elections_database/classic/Alaska_04102020_PRESIDENTOFTHEUNITEDSTATES.csv")

# Prepare data for histogram
data_list = [x for x, count in points.items() for _ in range(count)]

# Plot histogram
plt.figure(figsize=(5, 3))
plt.hist(data_list, bins=50, density=True, alpha=0.7)
plt.title('Histogram of Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

# Plot kernal density estimation
plt.figure(figsize=(5, 3))
sns.kdeplot(data_list, fill=True)
plt.title('Kernel Density Estimation of Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

In [ ]:
# Get the consistency points for the election
points = get_consistency_points("../rcv_elections_database/classic/NewYorkCity_06222021_DEMMayorCitywide.csv")

# Prepare data for histogram
data_list = [x for x, count in points.items() for _ in range(count)]

# Plot histogram
plt.figure(figsize=(5, 3))
plt.hist(data_list, bins=50, density=True, alpha=0.7)
plt.title('Histogram of Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

# Plot kernal density estimation
plt.figure(figsize=(5, 3))
sns.kdeplot(data_list, fill=True)
plt.title('Kernel Density Estimation of Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()